# Description

Combines all gene enrichment results found in input directory.

# Modules loading

In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from clustermatch import conf

# Settings

In [2]:
# ENRICH_FUNCTION = "enrichGO"

In [3]:
# CORRELATION_METHOD_NAME = "clustermatch"

In [4]:
# GENE_SELECTION_STRATEGY = "var_pc_log2"

In [5]:
# # clusterProfiler settings
# ENRICH_FUNCTION = "enrichGO"
# SIMPLIFY_CUTOFF = 0.7
# GO_ONTOLOGIES = ("BP", "CC", "MF")

In [6]:
# SIMILARITY_MATRICES_DIR = conf.GTEX["SIMILARITY_MATRICES_DIR"]
# display(SIMILARITY_MATRICES_DIR)

In [7]:
# SIMILARITY_MATRIX_FILENAME_TEMPLATE = conf.GTEX["SIMILARITY_MATRIX_FILENAME_TEMPLATE"]
# display(SIMILARITY_MATRIX_FILENAME_TEMPLATE)

# Paths

In [8]:
INPUT_DIR = conf.GTEX["GENE_ENRICHMENT_DIR"]
display(INPUT_DIR)
assert INPUT_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment')

In [9]:
OUTPUT_FILE = conf.GTEX["GENE_ENRICHMENT_COMBINED_FILE"]
display(OUTPUT_FILE)

OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data-gene_set_enrichment.pkl')

# Get data files

In [10]:
filename_pattern = re.compile(conf.GTEX["GENE_ENRICHMENT_FILENAME_PATTERN"])

In [11]:
# get input data files according to Settings
input_files = sorted(
    [
        f
        for f in INPUT_DIR.iterdir()
        if (m := re.search(filename_pattern, str(f))) is not None
        #         and m.group("corr_method") == CORRELATION_METHOD_NAME
    ]
)
display(len(input_files))
display(input_files[:5])

assert len(input_files) > 0

162

[PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data_adipose_subcutaneous-var_pc_log2-clustermatch-SpectralClustering-enrichGO-BP_full.pkl'),
 PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data_adipose_subcutaneous-var_pc_log2-clustermatch-SpectralClustering-enrichGO-BP_simplified_070.pkl'),
 PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data_adipose_subcutaneous-var_pc_log2-clustermatch-SpectralClustering-enrichGO-CC_full.pkl'),
 PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data_adipose_subcutaneous-var_pc_log2-clustermatch-SpectralClustering-enrichGO-CC_simplified_070.pkl'),
 PosixPath('/home/miltondp/projects/

## Preview data

In [12]:
display(input_files[0])

PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data_adipose_subcutaneous-var_pc_log2-clustermatch-SpectralClustering-enrichGO-BP_full.pkl')

In [13]:
_tmp_df = pd.read_pickle(input_files[0])

In [14]:
_tmp_df.shape

(50515, 12)

In [15]:
_tmp_df.sample(n=5, random_state=0)

,Cluster,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,clustering_id,clustering_n_clusters
17492,C21,GO:0035383,thioester metabolic process,15/329,34/3528,9.214483e-08,6.388708e-06,5.708822e-06,FASN/ACSL1/ACACB/ACOT1/ACSM5/ELOVL5/SLC25A1/AA...,15,SpectralClustering #13,65
25252,C60,GO:0032103,positive regulation of response to external st...,14/114,170/3528,9.718798e-04,1.609940e-02,1.372641e-02,CD209/CLEC10A/CSF1R/TYROBP/AIF1/CD28/P2RY12/BT...,14,SpectralClustering #16,80
38316,C1,GO:0071398,cellular response to fatty acid,3/20,19/3528,1.425567e-04,9.785604e-03,8.874987e-03,AKR1C2/AKR1C1/AKR1C3,3,SpectralClustering #21,125
3849,C3,GO:0045321,leukocyte activation,45/93,490/3528,5.381953e-16,1.085833e-15,5.068876e-16,IGHG2/IGKC/IGLC1/IGHG1/IGHG3/IGHM/IGHA1/IGLC3/...,45,SpectralClustering #4,20
26471,C36,GO:0060627,regulation of vesicle-mediated transport,3/7,129/3528,1.500944e-03,1.795773e-02,9.874634e-03,FGB/APOC3/FGG,3,SpectralClustering #16,80


# Run

## Read data, convert dtypes, add new metrics

In [16]:
all_results = []

for f_full in tqdm(input_files, ncols=100):
    f_name = f_full.name

    f_data = pd.read_pickle(f_full)
    f_data = f_data.rename(
        columns={
            "Count": "gene_count",
            "GeneRatio": "gene_ratio",
            "BgRatio": "bg_ratio",
            "ID": "go_term_id",
            "Description": "go_term_desc",
            "Cluster": "cluster_id",
            "clustering_n_clusters": "n_clusters",
            "p.adjust": "fdr",
        }
    )

    # genes in cluster
    f_data = f_data.assign(
        gene_total=f_data["gene_ratio"].apply(lambda x: int(x.split("/")[1]))
    )

    # background genes
    f_data = f_data.assign(
        bg_count=f_data["bg_ratio"].apply(lambda x: int(x.split("/")[0]))
    )
    f_data = f_data.assign(
        bg_total=f_data["bg_ratio"].apply(lambda x: int(x.split("/")[1]))
    )

    # add metadata
    metadata = re.search(filename_pattern, f_name)

    f_data = f_data[
        [
            "n_clusters",
            "cluster_id",
            "go_term_id",
            "go_term_desc",
            "gene_count",
            "gene_total",
            "gene_ratio",
            "bg_count",
            "bg_total",
            "bg_ratio",
            "fdr",
        ]
    ]

    f_data["tissue"] = metadata.group("tissue")
    f_data["gene_sel_strategy"] = metadata.group("gene_sel_strategy")
    f_data["corr_method"] = metadata.group("corr_method")
    f_data["clust_method"] = metadata.group("clust_method")
    f_data["enrich_func"] = metadata.group("enrich_func")
    f_data["results_subset"] = metadata.group("results_subset")

    all_results.append(f_data)

100%|█████████████████████████████████████████████████████████████| 162/162 [00:07<00:00, 21.72it/s]


In [17]:
df = pd.concat(all_results, ignore_index=True)

# to category dtype
df["cluster_id"] = df["cluster_id"].astype("category")
df["go_term_id"] = df["go_term_id"].astype("category")
df["go_term_desc"] = df["go_term_desc"].astype("category")
df["tissue"] = df["tissue"].astype("category")
df["gene_sel_strategy"] = df["gene_sel_strategy"].astype("category")
df["corr_method"] = df["corr_method"].astype("category")
df["clust_method"] = df["clust_method"].astype("category")
df["enrich_func"] = df["enrich_func"].astype("category")
df["results_subset"] = df["results_subset"].astype("category")

# convert to int32
df["n_clusters"] = df["n_clusters"].astype("int32")
df["gene_count"] = df["gene_count"].astype("int32")
df["gene_total"] = df["gene_total"].astype("int32")
df["bg_count"] = df["bg_count"].astype("int32")
df["bg_total"] = df["bg_total"].astype("int32")

# convert ratios to numbers
df["gene_ratio"] = df["gene_count"].div(df["gene_total"])
df["bg_ratio"] = df["bg_count"].div(df["bg_total"])

# add other metrics
df["rich_factor"] = df["gene_count"].div(df["bg_count"])
df["fold_enrich"] = df["gene_ratio"].div(df["bg_ratio"])

In [18]:
df.shape

(2505045, 19)

In [19]:
display(df.dtypes)
assert df.dtypes.loc["cluster_id"] == "category"

n_clusters              int32
cluster_id           category
go_term_id           category
go_term_desc         category
gene_count              int32
gene_total              int32
gene_ratio            float64
bg_count                int32
bg_total                int32
bg_ratio              float64
fdr                   float64
tissue               category
gene_sel_strategy    category
corr_method          category
clust_method         category
enrich_func          category
results_subset       category
rich_factor           float64
fold_enrich           float64
dtype: object

In [20]:
df.sample(n=5)

,n_clusters,cluster_id,go_term_id,go_term_desc,gene_count,gene_total,gene_ratio,bg_count,bg_total,bg_ratio,fdr,tissue,gene_sel_strategy,corr_method,clust_method,enrich_func,results_subset,rich_factor,fold_enrich
978402,200,C186,GO:0040029,"regulation of gene expression, epigenetic",6,67,0.089552,25,3278,0.007627,6.965386e-04,artery_tibial,var_pc_log2,spearman_full,SpectralClustering,enrichGO,BP_full,0.240000,11.742090
1304329,25,C18,GO:0050920,regulation of chemotaxis,18,363,0.049587,73,3746,0.019487,5.209219e-03,muscle_skeletal,var_pc_log2,spearman_abs,SpectralClustering,enrichGO,BP_simplified_070,0.246575,2.544549
1901493,95,C31,GO:0043043,peptide biosynthetic process,3,3,1.000000,61,3407,0.017904,1.174999e-04,skin_sun_exposed_lower_leg,var_pc_log2,spearman_full,SpectralClustering,enrichGO,BP_full,0.049180,55.852459
2250081,95,C6,GO:0007517,muscle organ development,9,33,0.272727,81,4013,0.020184,2.037367e-07,whole_blood,var_pc_log2,pearson_full,SpectralClustering,enrichGO,BP_simplified_070,0.111111,13.511785
1829702,85,C29,GO:1900182,positive regulation of protein localization to...,3,56,0.053571,16,3407,0.004696,8.008056e-03,skin_sun_exposed_lower_leg,var_pc_log2,spearman_abs,SpectralClustering,enrichGO,BP_simplified_070,0.187500,11.407366


## Some stats

In [21]:
display(df["fdr"].describe())
assert df["fdr"].min() > 0.0
assert df["fdr"].max() < 1.0

count     2.505045e+06
mean      1.346824e-02
std       1.561678e-02
min      6.807472e-203
25%       1.132908e-04
50%       5.758670e-03
75%       2.507374e-02
max       4.999997e-02
Name: fdr, dtype: float64

In [22]:
df["n_clusters"].unique()

array([  2,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,
        65,  70,  75,  80,  85,  90,  95, 100, 125, 150, 175, 200],
      dtype=int32)

In [23]:
df["tissue"].unique()

['adipose_subcutaneous', 'artery_tibial', 'muscle_skeletal', 'skin_sun_exposed_lower_leg', 'whole_blood']
Categories (5, object): ['adipose_subcutaneous', 'artery_tibial', 'muscle_skeletal', 'skin_sun_exposed_lower_leg', 'whole_blood']

In [24]:
df["gene_sel_strategy"].unique()

['var_pc_log2']
Categories (1, object): ['var_pc_log2']

In [25]:
df["corr_method"].unique()

['clustermatch', 'clustermatch_k2', 'pearson_abs', 'pearson_full', 'spearman_abs', 'spearman_full']
Categories (6, object): ['clustermatch', 'clustermatch_k2', 'pearson_abs', 'pearson_full', 'spearman_abs', 'spearman_full']

In [26]:
df["clust_method"].unique()

['SpectralClustering']
Categories (1, object): ['SpectralClustering']

In [27]:
df["results_subset"].unique()

['BP_full', 'BP_simplified_070', 'CC_full', 'CC_simplified_070', 'MF_full', 'MF_simplified_070']
Categories (6, object): ['BP_full', 'BP_simplified_070', 'CC_full', 'CC_simplified_070', 'MF_full', 'MF_simplified_070']

## Testing

In [28]:
assert not df.isna().any().any()

In [29]:
# test if values are correctly calculated
_tmp = df[
    (df.go_term_id == "GO:0035383")
    & (df.n_clusters == 65)
    & (df.cluster_id == "C21")
    & (df.tissue == "adipose_subcutaneous")
    & (df.gene_sel_strategy == "var_pc_log2")
    & (df.corr_method == "clustermatch")
    & (df.clust_method == "SpectralClustering")
    & (df.enrich_func == "enrichGO")
    & (df.results_subset == "BP_full")
]
assert _tmp.shape[0] == 1
_tmp = _tmp.iloc[0]

assert _tmp["gene_count"] == 15
assert _tmp["gene_total"] == 329
assert _tmp["gene_ratio"] == 15.0 / 329.0
assert _tmp["bg_count"] == 34
assert _tmp["bg_total"] == 3528
assert _tmp["bg_ratio"] == 34.0 / 3528.0
assert _tmp["rich_factor"] == 15.0 / 34.0
assert _tmp["fold_enrich"] == (15.0 / 329.0) / (34.0 / 3528.0)

# Save

In [30]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

saveRDS = ro.r["saveRDS"]
readRDS = ro.r["readRDS"]

In [31]:
data = df

## Pickle

In [32]:
display(OUTPUT_FILE)

PosixPath('/home/miltondp/projects/labs/greenelab/clustermatch_repos/clustermatch-gene-expr/base/results/gtex_v8/gene_set_enrichment/gtex_v8_data-gene_set_enrichment.pkl')

In [33]:
data.to_pickle(OUTPUT_FILE)

## RDS

In [ ]:
output_rds_file = OUTPUT_FILE.with_suffix(".rds")
display(output_rds_file)

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_r = ro.conversion.py2rpy(data)

In [ ]:
data_r

In [ ]:
saveRDS(data_r, str(output_rds_file))

In [ ]:
# testing
data_r = readRDS(str(output_rds_file))

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_again = ro.conversion.rpy2py(data_r)

    # convert index to int, otherwise it's converted to string
    data_again.index = data_again.index.astype(int)

In [ ]:
data_again.shape

In [ ]:
data_again.head()

In [ ]:
pd.testing.assert_frame_equal(
    data,
    data_again,
    check_names=False,  # do not check "name" attribute of index and column
    check_exact=True,  # since this is a binary format, it should match exactly
)

## tsv.gz

In [ ]:
output_text_file = OUTPUT_FILE.with_suffix(".tsv.gz")
display(output_text_file)

In [ ]:
data.to_csv(output_text_file, sep="\t", index=False, float_format="%.5e")

In [ ]:
# testing
data_again = pd.read_csv(output_text_file, sep="\t")  # , index_col=0)

In [ ]:
data_again.shape

In [ ]:
data_again.head()

In [ ]:
pd.testing.assert_frame_equal(
    data,
    data_again,
    check_names=False,  # do not check "name" attribute of index and column
    check_dtype=False,  # do not check dtypes: do not distinguish between int64 and int32, for instance
    check_categorical=False,
    check_exact=False,
    rtol=1e-5,
    atol=5e-5,
)